In [1]:
import skimage.io
import dask.array as da
import dask
from pathlib import Path
import napari
from natsort import natsorted
import numpy as np
from napari_flim_phasor_calculator._reader import get_current_tz

Define read 2d and 3d functions

In [2]:

# folder_path = r"C:\Users\mazo260d\Desktop\Conni_BiA_PoL\embryo_FLIM_data\raw_data_embryo_stack_3tps_43pl_2ch\output_as_tif"
# folder_path = r"C:\Users\mazo260d\Desktop\Conni_BiA_PoL\stack_smaller_as_tif"
folder_path = r"C:\Users\mazo260d\Desktop\Conni_BiA_PoL\stack_as_tif"

folder_path = Path(folder_path)

def read_tif_data_2D(path):
    from skimage.io import imread
    image = imread(path)
    return image

def read_tif_data_3D(file_paths, image_slice_max_shape):
    file_paths = natsorted(file_paths)
    z_list = []
    for file_path in file_paths:
        if file_path.suffix == '.tif':
            current_t, current_z = get_current_tz(file_path)
            data = read_tif_data_2D(file_path)
            if current_z is not None:
                image = np.zeros(image_slice_max_shape, dtype=data.dtype)
                image[:data.shape[0], :data.shape[1], :data.shape[2], :data.shape[3]] = data
                z_list.append(image)
    image_3D = np.stack(z_list)
    # move channel and microtime to the beginning (putting z behind them): from (z, ch, mt, y, x) to (ch, mt, z, y, x)
    image_3D = np.moveaxis(image_3D, [-4, -3], [0, 1])
    return image_3D

Get max slice shape

In [3]:
# Read all slices to get slice max shape and dtype (not ideal but for now it is OK)
slice_shape_list = []
for file_path in folder_path.iterdir():
    if file_path.suffix == '.tif':
        image_2D = read_tif_data_2D(file_path)
        slice_shape_list.append(image_2D.shape)
slice_max_shape = max(slice_shape_list)
print('last_slice_shape = ', image_2D.shape,'image_dtype = ', image_2D.dtype)
print('max_slice_shape = ', slice_max_shape)

last_slice_shape =  (2, 266, 512, 512) image_dtype =  uint16
max_slice_shape =  (2, 276, 512, 512)


Get max z slices and max t timepoints from filepaths

In [4]:
file_paths = [file_path for file_path in folder_path.iterdir() if file_path.suffix == '.tif']

# Get max z slices by reading all file names
def get_max_zslices(file_paths):
    max_z = max([get_current_tz(file_path) for file_path in file_paths if file_path.suffix == '.tif'])[1]
    if max_z is None:
        return 1
    return max_z
def get_max_time(file_paths):
    max_time = max([get_current_tz(file_path) for file_path in file_paths if file_path.suffix == '.tif'])[0]
    if max_time is None:
        return 1
    return max_time

max_z = get_max_zslices(file_paths)
print('max_z slices = ', max_z)
max_t = get_max_time(file_paths)
print('max_time = ', max_t)

max_z slices =  65
max_time =  1


Get stack max shape

In [5]:
image_stack_shape = (*slice_max_shape[:-2], max_z, *slice_max_shape[-2:])
image_stack_shape


(2, 276, 65, 512, 512)

Make dask array

In [6]:
t_path_list = []
z_path_list = []
file_paths = natsorted(file_paths)
previous_t = 1
for file_path in file_paths:
    if file_path.suffix == '.tif':
        current_t, current_z = get_current_tz(file_path)
        print(current_t, current_z)
        if current_t is not None:
            if current_t > previous_t:
                t_path_list.append(z_path_list)
                z_path_list = []
                previous_t = current_t
            z_path_list.append(file_path)

# If no timepoints, z+path_list is file_paths
if current_t is None:
    z_path_list = file_paths

# Append last timepoint
t_path_list.append(z_path_list)


None 1
None 2
None 3
None 4
None 5
None 6
None 7
None 8
None 9
None 10
None 11
None 12
None 13
None 14
None 15
None 16
None 17
None 18
None 19
None 20
None 21
None 22
None 23
None 24
None 25
None 26
None 27
None 28
None 29
None 30
None 31
None 32
None 33
None 34
None 35
None 36
None 37
None 38
None 39
None 40
None 41
None 42
None 43
None 44
None 45
None 46
None 47
None 48
None 49
None 50
None 51
None 52
None 53
None 54
None 55
None 56
None 57
None 58
None 59
None 60
None 61
None 62
None 63
None 64
None 65


In [7]:
z_path_list

[WindowsPath('C:/Users/mazo260d/Desktop/Conni_BiA_PoL/stack_as_tif/FLIM_stack_z1.tif'),
 WindowsPath('C:/Users/mazo260d/Desktop/Conni_BiA_PoL/stack_as_tif/FLIM_stack_z2.tif'),
 WindowsPath('C:/Users/mazo260d/Desktop/Conni_BiA_PoL/stack_as_tif/FLIM_stack_z3.tif'),
 WindowsPath('C:/Users/mazo260d/Desktop/Conni_BiA_PoL/stack_as_tif/FLIM_stack_z4.tif'),
 WindowsPath('C:/Users/mazo260d/Desktop/Conni_BiA_PoL/stack_as_tif/FLIM_stack_z5.tif'),
 WindowsPath('C:/Users/mazo260d/Desktop/Conni_BiA_PoL/stack_as_tif/FLIM_stack_z6.tif'),
 WindowsPath('C:/Users/mazo260d/Desktop/Conni_BiA_PoL/stack_as_tif/FLIM_stack_z7.tif'),
 WindowsPath('C:/Users/mazo260d/Desktop/Conni_BiA_PoL/stack_as_tif/FLIM_stack_z8.tif'),
 WindowsPath('C:/Users/mazo260d/Desktop/Conni_BiA_PoL/stack_as_tif/FLIM_stack_z9.tif'),
 WindowsPath('C:/Users/mazo260d/Desktop/Conni_BiA_PoL/stack_as_tif/FLIM_stack_z10.tif'),
 WindowsPath('C:/Users/mazo260d/Desktop/Conni_BiA_PoL/stack_as_tif/FLIM_stack_z11.tif'),
 WindowsPath('C:/Users/mazo260

In [8]:
for z_path_list in t_path_list:
    print(z_path_list)

[WindowsPath('C:/Users/mazo260d/Desktop/Conni_BiA_PoL/stack_as_tif/FLIM_stack_z1.tif'), WindowsPath('C:/Users/mazo260d/Desktop/Conni_BiA_PoL/stack_as_tif/FLIM_stack_z2.tif'), WindowsPath('C:/Users/mazo260d/Desktop/Conni_BiA_PoL/stack_as_tif/FLIM_stack_z3.tif'), WindowsPath('C:/Users/mazo260d/Desktop/Conni_BiA_PoL/stack_as_tif/FLIM_stack_z4.tif'), WindowsPath('C:/Users/mazo260d/Desktop/Conni_BiA_PoL/stack_as_tif/FLIM_stack_z5.tif'), WindowsPath('C:/Users/mazo260d/Desktop/Conni_BiA_PoL/stack_as_tif/FLIM_stack_z6.tif'), WindowsPath('C:/Users/mazo260d/Desktop/Conni_BiA_PoL/stack_as_tif/FLIM_stack_z7.tif'), WindowsPath('C:/Users/mazo260d/Desktop/Conni_BiA_PoL/stack_as_tif/FLIM_stack_z8.tif'), WindowsPath('C:/Users/mazo260d/Desktop/Conni_BiA_PoL/stack_as_tif/FLIM_stack_z9.tif'), WindowsPath('C:/Users/mazo260d/Desktop/Conni_BiA_PoL/stack_as_tif/FLIM_stack_z10.tif'), WindowsPath('C:/Users/mazo260d/Desktop/Conni_BiA_PoL/stack_as_tif/FLIM_stack_z11.tif'), WindowsPath('C:/Users/mazo260d/Desktop/C

In [9]:
read_tif_data_3D_delayed = dask.delayed(read_tif_data_3D, pure=True)

lazy_images = [read_tif_data_3D_delayed(z_path_list, image_slice_max_shape=slice_max_shape) for z_path_list in t_path_list]

arrays = [da.from_delayed(lazy_image,           # Construct a small Dask array
                          dtype=image_2D.dtype,   # for every lazy value
                          shape=image_stack_shape)
          for lazy_image in lazy_images]

stack = da.stack(arrays, axis=0)                # Stack all small Dask arrays into one
print('stack shape and chunks = ', stack.shape, '\n', stack.chunks)

stack shape and chunks =  (1, 2, 276, 65, 512, 512) 
 ((1,), (2,), (276,), (65,), (512,), (512,))


In [10]:
stack = da.moveaxis(stack, 0, -4)

In [11]:
stack

dask.array<transpose, shape=(2, 276, 1, 65, 512, 512), dtype=uint16, chunksize=(2, 276, 1, 65, 512, 512), chunktype=numpy.ndarray>

In [12]:
stack.chunksize

(2, 276, 1, 65, 512, 512)

In [13]:
np.cumprod(stack.chunksize)

array([        2,       552,       552,     35880,  18370560, 815792128])

In [14]:
stack.itemsize

2

In [15]:
stack.chunks

((2,), (276,), (1,), (65,), (512,), (512,))

In [16]:
chunk_size_MBytes = np.cumprod(stack.chunksize)[-1] * stack.itemsize / 1e6 #MB per chunk
chunk_size_MBytes

1631.584256

In [17]:
if chunk_size_MBytes > 1e3: # if larger than 1GB (1000MB)
    stack = stack.rechunk({-3: 'auto'}, block_size_limit=1e9)

In [18]:
stack

dask.array<rechunk-merge, shape=(2, 276, 1, 65, 512, 512), dtype=uint16, chunksize=(2, 276, 1, 3, 512, 512), chunktype=numpy.ndarray>

In [19]:
viewer = napari.Viewer()

In [20]:
viewer.add_image(stack, channel_axis=0)